<a href="https://colab.research.google.com/github/Jeyharshini10/Infosys_Springboard/blob/main/milestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvidia-smi


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!pip install transformers datasets sentencepiece rouge-score
!pip install nltk



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import textwrap
import re
import requests
import ssl
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

print(" Libraries imported.")

We will use two online text sources for this experiment:

1. Alice in Wonderland  
   URL: `https://www.gutenberg.org/files/11/11-0.txt`

2. Dracula  
   URL: `https://www.gutenberg.org/files/345/345-0.txt`

We limit each text to the first 3000 characters for faster processing.


In [ ]:
import requests

url1 = "https://www.gutenberg.org/files/11/11-0.txt"   # Alice in Wonderland
url2 = "https://www.gutenberg.org/files/345/345-0.txt"  # Dracula

text1 = requests.get(url1).text[:3000]  # limit to first 3000 characters
text2 = requests.get(url2).text[:3000]

documents = [text1, text2]
print("Documents fetched successfully!")


Step 2: T5 Summarization & Paraphrasing

The T5 model converts all NLP tasks into a text-to-text format  
We will first summarize the document, then perform paraphrasing on the summary using a T5 paraphrase model.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# T5 Summarization
def summarize_t5(text, min_len=50, max_len=150):
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    input_text = "summarize: " + text.strip().replace("\n"," ")
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs, max_length=max_len, min_length=min_len, num_beams=4, length_penalty=2.0, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# T5 Paraphrasing
def paraphrase_t5(text):
    model_name = "Vamsi/T5_Paraphrase_Paws"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    input_text = "paraphrase: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    paraphrase_ids = model.generate(inputs, max_length=256, num_beams=4, early_stopping=True)
    return tokenizer.decode(paraphrase_ids[0], skip_special_tokens=True)

# Run T5 on documents
for idx, doc in enumerate(documents):
    print(f"\n=== Document {idx+1} – T5 ===")
    t5_summary = summarize_t5(doc)
    t5_paraphrase = paraphrase_t5(t5_summary)
    print("\nT5 Summary:\n", t5_summary)
    print("\nT5 Paraphrase:\n", t5_paraphrase)


 Load and Clean Text from a URL

We will fetch a text file from a URL and clean it by:
1. Removing unnecessary newlines.
2. Removing extra spaces.
3. Keeping only readable text.


In [ ]:
import requests
import re

# URL of the text file
url = "https://www.gutenberg.org/files/11/11-0.txt"  # Alice in Wonderland

# Fetch the text
response = requests.get(url)
raw_text = response.text

# Clean the text
# Replace multiple spaces, tabs, and newlines with a single space
clean_text = re.sub(r'\s+', ' ', raw_text).strip()

# Optional: limit to first 3000 characters for faster processing
clean_text = clean_text[:3000]

# Preview the first 500 characters
print(clean_text[:500])


Step 3: BART Summarization

The BART model is bidirectional and autoregressive, useful for abstractive summarization tasks.  
We will summarize the same documents using BART for comparison.


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

def summarize_bart(text, min_len=50, max_len=150):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'], max_length=max_len, min_length=min_len, num_beams=4, length_penalty=2.0, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

for idx, doc in enumerate(documents):
    print(f"\n=== Document {idx+1} – BART ===")
    bart_summary = summarize_bart(doc)
    print("\nBART Summary:\n", bart_summary)


Step 4: Pegasus Summarization

Pegasus is pre-trained specifically for **abstractive summarization** by predicting missing sentences in documents.  
We will summarize the same documents using Pegasus for comparison.


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

def summarize_pegasus(text, min_len=50, max_len=150):
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'], max_length=max_len, min_length=min_len, num_beams=4, length_penalty=2.0, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

for idx, doc in enumerate(documents):
    print(f"\n=== Document {idx+1} – Pegasus ===")
    pegasus_summary = summarize_pegasus(doc)
    print("\nPegasus Summary:\n", pegasus_summary)


Step 5: Model Comparison

We compare the **summary lengths** (number of words) produced by each model.  
A heatmap will visualize which model produces longer or shorter summaries for each document.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

summary_lengths = {"T5": [], "BART": [], "Pegasus": []}

for doc in documents:
    summary_lengths["T5"].append(len(summarize_t5(doc).split()))
    summary_lengths["BART"].append(len(summarize_bart(doc).split()))
    summary_lengths["Pegasus"].append(len(summarize_pegasus(doc).split()))

heatmap_data = [
    summary_lengths["T5"],
    summary_lengths["BART"],
    summary_lengths["Pegasus"]
]

model_names = ["T5", "BART", "Pegasus"]
document_names = [f"Doc {i+1}" for i in range(len(documents))]

plt.figure(figsize=(8,4))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="YlGnBu", xticklabels=document_names, yticklabels=model_names)
plt.title("Heatmap of Summary Lengths Across Models and Documents")
plt.xlabel("Documents")
plt.ylabel("Models")
plt.show()



In this example, we will:

1. Load a text file from a URL.
2. Clean the text.
3. Summarize it using T5, BART, and Pegasus.
4. Compare summary lengths using a bar chart.


In [ ]:
import requests
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import matplotlib.pyplot as plt

# -------------------------------
# Step 1: Load and clean text
# -------------------------------
url = "https://www.gutenberg.org/files/11/11-0.txt"  # Alice in Wonderland
response = requests.get(url)
raw_text = response.text

# Clean text: remove extra spaces, newlines, tabs
clean_text = re.sub(r'\s+', ' ', raw_text).strip()
# Optional: limit length for faster processing
clean_text = clean_text[:3000]

# -------------------------------
# Step 2: Define summarization functions
# -------------------------------
def summarize_t5(text):
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_bart(text):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_pegasus(text):
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# -------------------------------
# Step 3: Generate summaries
# -------------------------------
t5_summary = summarize_t5(clean_text)
bart_summary = summarize_bart(clean_text)
pegasus_summary = summarize_pegasus(clean_text)

# Print summaries
print("T5 Summary:\n", t5_summary, "\n")
print("BART Summary:\n", bart_summary, "\n")
print("Pegasus Summary:\n", pegasus_summary, "\n")

# -------------------------------
# Step 4: Compare summary lengths
# -------------------------------
summary_lengths = {
    "T5": len(t5_summary.split()),
    "BART": len(bart_summary.split()),
    "Pegasus": len(pegasus_summary.split())
}

# -------------------------------
# Step 5: Plot bar chart
# -------------------------------
plt.figure(figsize=(6,4))
plt.bar(summary_lengths.keys(), summary_lengths.values(), color=['skyblue','lightgreen','salmon'])
plt.title("Summary Length Comparison")
plt.ylabel("Number of Words")
plt.show()


T5 Paraphrasing

We will use a pretrained T5 paraphrase model to generate a paraphrased version of the text.  
This helps in generating alternative expressions while keeping the meaning intact.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Function to paraphrase text using T5
def paraphrase_t5(text):
    model_name = "Vamsi/T5_Paraphrase_Paws"  # Pretrained T5 paraphrase model
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Prepare input for the model
    input_text = "paraphrase: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate paraphrase
    paraphrase_ids = model.generate(inputs, max_length=256, num_beams=4, early_stopping=True)
    paraphrased_text = tokenizer.decode(paraphrase_ids[0], skip_special_tokens=True)
    return paraphrased_text

# Example usage
original_text = "Alice was beginning to get very tired of sitting by her sister on the bank."
paraphrased_text = paraphrase_t5(original_text)

print("Original Text:\n", original_text)
print("\nParaphrased Text:\n", paraphrased_text)


 Pegasus Paraphrasing

Although Pegasus is mainly used for summarization, it can be used to generate paraphrased versions of text in an abstractive way.


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Function to "paraphrase" text using Pegasus
def paraphrase_pegasus(text, max_len=100, min_len=30):
    # Use the pre-trained Pegasus XSUM model
    model_name = "google/pegasus-xsum"
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)

    # Encode the text
    inputs = tokenizer(text, return_tensors="pt", truncation=True)

    # Generate output
    paraphrase_ids = model.generate(
        inputs['input_ids'],
        max_length=max_len,
        min_length=min_len,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode output
    paraphrased_text = tokenizer.decode(paraphrase_ids[0], skip_special_tokens=True)
    return paraphrased_text

# Example usage
original_text = "Alice was beginning to get very tired of sitting by her sister on the bank."
pegasus_paraphrase = paraphrase_pegasus(original_text)

print("Original Text:\n", original_text)
print("\nPegasus Paraphrased Text:\n", pegasus_paraphrase)
